<a href="https://colab.research.google.com/github/vviia/ml-identification-of-hypertension/blob/main/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### import library

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import find_peaks
from scipy import signal
from scipy.integrate import trapz
from scipy.interpolate import interp1d
import seaborn as sns

In [ ]:
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

from sklearn.model_selection import train_test_split # for splitting the data into train and test samples
from sklearn.metrics import classification_report # for model evaluation metrics
from sklearn.svm import SVC # for Support Vector Classification model

import plotly.express as px  # for data visualization
import plotly.graph_objects as go # for data visualization

### load data frame

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dff = pd.read_csv('/content/drive/MyDrive/final-project-tesis/DATA/DATA_NEW.csv', encoding='utf-8')
df1 = dff[['HR_KUBIOS',	'HR_BEATS',	'RR_MS', 'RMSSD',	'STD_HR',	'STD_RR','VLF_MS2','LF_MS2', 'TOTAL_MS2','LF_PER_HF','VLF_HZ','LF_HZ','HF_HZ','LF_NU','HF_NU', 'class'	]]
df1.head()

,HR_KUBIOS,HR_BEATS,RR_MS,RMSSD,STD_HR,STD_RR,VLF_MS2,LF_MS2,TOTAL_MS2,LF_PER_HF,VLF_HZ,LF_HZ,HF_HZ,LF_NU,HF_NU,class
0,58.770,58.77,1021.66,30.930,1.731,28.33,1.32,2.96,7.17,1.03,0.03,0.06,0.31,50.66,49.34,1
1,60.310,60.48,994.72,62.190,3.270,49.91,0.61,2.47,14.28,0.22,0.03,0.14,0.17,18.10,81.90,1
2,61.114,61.17,981.76,26.855,1.820,29.94,0.53,6.37,8.16,5.03,0.03,0.09,0.22,83.43,16.57,1
3,59.710,59.71,1004.78,10.870,0.540,9.26,0.00,0.42,0.59,2.43,0.00,0.09,0.31,70.87,29.13,1
4,77.480,77.48,774.34,7.970,0.490,4.95,0.00,0.01,0.09,0.13,0.00,0.06,0.35,11.40,88.60,1


### normalisasi

In [ ]:
#function of normalization
def norm (x):
  #from sklearn import preprocessing
  import numpy as np
  hasil = (x-x.min())/(x.max()-x.min())
  return(hasil)

In [ ]:
# normalisasi data
df = norm(df1)
# menampilkan data
df.head()

,HR_KUBIOS,HR_BEATS,RR_MS,RMSSD,STD_HR,STD_RR,VLF_MS2,LF_MS2,TOTAL_MS2,LF_PER_HF,VLF_HZ,LF_HZ,HF_HZ,LF_NU,HF_NU,class
0,0.144780,0.206826,0.499039,0.019620,0.010583,0.023927,0.025162,0.013954,0.007428,0.055948,0.75,0.181818,0.64,0.534106,0.465894,1.0
1,0.153873,0.213289,0.480546,0.039449,0.019991,0.042154,0.011628,0.011644,0.014793,0.011950,0.75,0.909091,0.08,0.190828,0.809172,1.0
2,0.158621,0.215897,0.471650,0.017035,0.011127,0.025287,0.010103,0.030030,0.008453,0.273221,0.75,0.454545,0.28,0.879599,0.120401,1.0
3,0.150331,0.210379,0.487452,0.006895,0.003301,0.007821,0.000000,0.001980,0.000611,0.131993,0.00,0.454545,0.64,0.747180,0.252820,1.0
4,0.255255,0.277545,0.329270,0.005056,0.002996,0.004181,0.000000,0.000047,0.000093,0.007061,0.00,0.181818,0.80,0.120190,0.879810,1.0


In [ ]:
df.min()

HR_KUBIOS    0.0
HR_BEATS     0.0
RR_MS        0.0
RMSSD        0.0
STD_HR       0.0
STD_RR       0.0
VLF_MS2      0.0
LF_MS2       0.0
TOTAL_MS2    0.0
LF_PER_HF    0.0
VLF_HZ       0.0
LF_HZ        0.0
HF_HZ        0.0
LF_NU        0.0
HF_NU        0.0
class        0.0
dtype: float64

In [ ]:
df.max()

HR_KUBIOS    1.0
HR_BEATS     1.0
RR_MS        1.0
RMSSD        1.0
STD_HR       1.0
STD_RR       1.0
VLF_MS2      1.0
LF_MS2       1.0
TOTAL_MS2    1.0
LF_PER_HF    1.0
VLF_HZ       1.0
LF_HZ        1.0
HF_HZ        1.0
LF_NU        1.0
HF_NU        1.0
class        1.0
dtype: float64

In [ ]:
dfx1 = df[['HR_KUBIOS',	'HR_BEATS',	'RR_MS', 'RMSSD',	'STD_HR',	'STD_RR','VLF_MS2','LF_MS2', 'TOTAL_MS2','LF_PER_HF','VLF_HZ','LF_HZ','HF_HZ','LF_NU','HF_NU']]
dfy = df[['class']].values

In [ ]:
da = df[[ 'HR_KUBIOS','HR_BEATS','STD_HR', 'RMSSD','STD_RR','VLF_MS2','VLF_HZ']]
da.describe()

,HR_KUBIOS,HR_BEATS,STD_HR,RMSSD,STD_RR,VLF_MS2,VLF_HZ
count,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000
mean,0.217886,0.259321,0.039940,0.048839,0.046361,0.033776,0.522936
std,0.150038,0.110431,0.088878,0.072959,0.069502,0.109206,0.396162
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.139215,0.203500,0.004218,0.007398,0.007092,0.000000,0.000000
50%,0.178673,0.230431,0.009690,0.020213,0.021030,0.001334,0.750000
75%,0.255270,0.280011,0.041343,0.072157,0.065929,0.017775,0.750000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### build model

In [ ]:
def fitting(X, y, C, gamma):
    # Create training and testing samples
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Fit the model
    # Note, available kernels: {‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
    model = SVC(kernel='rbf', probability=True, C=C, gamma=gamma) #dicoba sigmoid
    clf = model.fit(X_train, y_train)

    # Predict class labels on training data
    pred_labels_tr = model.predict(X_train)
    # Predict class labels on a test data
    pred_labels_te = model.predict(X_test)

    # Use score method to get accuracy of the model
    print('----- Evaluation on Test Data -----')
    score_te = model.score(X_test, y_test)
    print('Accuracy Score: ', score_te)
    # Look at classification report to evaluate the model
    print(classification_report(y_test, pred_labels_te))
    print('--------------------------------------------------------')

    print('----- Evaluation on Training Data -----')
    score_tr = model.score(X_train, y_train)
    print('Accuracy Score: ', score_tr)
    # Look at classification report to evaluate the model
    print(classification_report(y_train, pred_labels_tr))
    print('--------------------------------------------------------')

    '''print(pred_labels_tr)
    print(pred_labels_te)
    col1 = "pred-ytrain"
    col2 = "pred-ytest"
    df = pd.DataFrame({col1:pred_labels_tr})
    df.to_csv('pred-train-rbf', index=False)
    df1 = pd.DataFrame({col2:pred_labels_te})
    df1.to_csv('pred-test-rbf', index=False)'''

    # Return relevant data for chart plotting
    return X_train, X_test, y_train, y_test, clf

    

### fitting data

In [ ]:
# Select data for model
#X = df[[ 'HR_KUBIOS','HR_BEATS','RMSSD','STD_HR','STD_RR','VLF_MS2','VLF_HZ','LF_NU','HF_NU']]
X = df[[ 'HR_KUBIOS','HR_BEATS','STD_HR', 'RMSSD','STD_RR','VLF_MS2','VLF_HZ']]

#X=df.drop('class', axis=1, inplace=True)
y= dfy.ravel()

# Fit the model and display results
X_train, X_test, y_train, y_test, clf = fitting(X, y, 100, 50000)

----- Evaluation on Test Data -----
Accuracy Score:  0.8068181818181818
              precision    recall  f1-score   support

         0.0       0.91      0.57      0.70        35
         1.0       0.77      0.96      0.86        53

    accuracy                           0.81        88
   macro avg       0.84      0.77      0.78        88
weighted avg       0.83      0.81      0.80        88

--------------------------------------------------------
----- Evaluation on Training Data -----
Accuracy Score:  0.9885057471264368
              precision    recall  f1-score   support

         0.0       1.00      0.97      0.99       137
         1.0       0.98      1.00      0.99       211

    accuracy                           0.99       348
   macro avg       0.99      0.99      0.99       348
weighted avg       0.99      0.99      0.99       348

--------------------------------------------------------
